In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Stroke"
cohort = "GSE273225"

# Input paths
in_trait_dir = "../../input/GEO/Stroke"
in_cohort_dir = "../../input/GEO/Stroke/GSE273225"

# Output paths
out_data_file = "../../output/preprocess/Stroke/GSE273225.csv"
out_gene_data_file = "../../output/preprocess/Stroke/gene_data/GSE273225.csv"
out_clinical_data_file = "../../output/preprocess/Stroke/clinical_data/GSE273225.csv"
json_path = "../../output/preprocess/Stroke/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"The effect of rewarming ischemia on tissue transcriptome signatures: a clinical observational study in lung transplantation"
!Series_summary	"BACKGROUND: In lung transplantation (LuTx), various ischemic phases exist, yet the rewarming ischemia time (RIT) during implantation has often been overlooked. During RIT, lungs are deflated and exposed to the body temperature in the recipient's chest cavity. Our prior clinical findings demonstrated that prolonged RIT increases the risk of primary graft dysfunction. However, the molecular mechanisms of rewarming ischemic injury in this context remain unexplored. We aimed to characterize the rewarming ischemia phase during LuTx by measuring organ temperature and comparing transcriptome and metabolome profiles in tissue obtained at the end versus the start of implantation."
!Series_summary	"METHODS: In a clinical observational study, 34 double-LuTx with ice preservation were analyzed. Lung core and surface tem

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# From the background information, this dataset contains transcriptome data from the nCounter Immunology V2 panel
# which targets 579 immune system-associated genes. This is gene expression data.
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# Looking for stroke-related variables in the sample characteristics
# From reviewing the data, this dataset is about lung transplantation, not stroke
# The rewarming ischemia time could potentially be used as a continuous variable related to ischemia
# but it doesn't provide a binary stroke/non-stroke classification
trait_row = None  # No Stroke variable available

# Age data is available in row 3
age_row = 3

# Gender data is available in row 4
gender_row = 4

# 2.2 Data Type Conversion

# Since trait is not available, we'll define a function but it won't be used
def convert_trait(value):
    return None

# Convert age (extract numeric value after colon)
def convert_age(value):
    if not value or ':' not in value:
        return None
    try:
        age_str = value.split(':')[1].strip()
        return float(age_str)
    except:
        return None

# Convert gender (female=0, male=1)
def convert_gender(value):
    if not value or ':' not in value:
        return None
    gender_str = value.split(':')[1].strip().lower()
    if 'female' in gender_str:
        return 0
    elif 'male' in gender_str:
        return 1
    else:
        return None

# 3. Save Metadata
# Initial filtering - this dataset doesn't have stroke trait data
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is None, we'll skip this step


False

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


Matrix file found: ../../input/GEO/Stroke/GSE273225/GSE273225_series_matrix.txt.gz
Gene data shape: (608, 118)
First 20 gene/probe identifiers:
Index(['ABCB1', 'ABCF1', 'ABL1', 'ADA', 'AHR', 'AICDA', 'AIRE', 'ALAS1', 'APP',
       'ARG1', 'ARG2', 'ARHGDIB', 'ATG10', 'ATG12', 'ATG16L1', 'ATG5', 'ATG7',
       'ATM', 'B2M', 'B3GAT1'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# These appear to be standard human gene symbols
# They include well-known genes like ABCB1 (multidrug resistance gene), 
# B2M (Beta-2-microglobulin), and ATM (ataxia telangiectasia mutated)
# No mapping to gene symbols is required as these are already gene symbols

requires_gene_mapping = False


### Step 5: Data Normalization and Linking

In [6]:
# 1. Normalize gene symbols in the obtained gene expression data
# Note: Gene symbols were already normalized in Step 6, but we'll do it again to ensure consistency
print(f"Original gene data shape: {gene_data.shape}")

# Normalize gene symbols
gene_data_normalized = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {gene_data_normalized.shape}")

# Save the normalized gene expression data if not already done
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data_normalized.to_csv(out_gene_data_file)
print(f"Normalized gene expression data saved to {out_gene_data_file}")

# 2. Create clinical data with the trait information (all samples are stroke patients)
# Extract sample IDs from gene data columns
sample_ids = gene_data_normalized.columns.tolist()

# Create a DataFrame with the Stroke trait (all samples are stroke patients)
clinical_data = pd.DataFrame({
    trait: [1] * len(sample_ids)
}, index=sample_ids)

# Transpose clinical_data to have the right format for linking
clinical_data = clinical_data.T

print(f"Created clinical data with shape: {clinical_data.shape}")
print("Clinical data sample:")
print(clinical_data.iloc[:, :5])  # Show first 5 columns

# Link clinical and genetic data
linked_data = pd.concat([clinical_data, gene_data_normalized], axis=0)
print(f"Linked data shape: {linked_data.shape}")
print("Linked data preview (first 5 rows, 5 columns):")
print(linked_data.iloc[:5, :5])

# 3. Handle missing values
# Note: Since we created the clinical data manually, there shouldn't be missing values in the trait column
linked_data_clean = linked_data.copy()

# 4. Check for bias in the dataset
# Since all samples have stroke (trait=1), this is a single-class dataset
# which is inherently biased for case-control studies
is_biased = True
print("\nDataset contains only one class for Stroke (all patients have stroke). This makes it biased for case-control studies.")

# 5. Conduct final quality validation
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=True,  # Trait data is available (all samples are stroke patients)
    is_biased=is_biased,
    df=linked_data_clean,
    note="Dataset contains gene expression data from peripheral blood of stroke patients. All samples are stroke patients (trait=1), which makes it biased for case-control studies but might be useful for other analyses beyond case-control comparisons."
)

# 6. Save the linked data if it's usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data_clean.to_csv(out_data_file, index=True)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset deemed not usable for case-control studies. Linked data not saved.")

Original gene data shape: (608, 118)
Gene data shape after normalization: (576, 118)
Normalized gene expression data saved to ../../output/preprocess/Stroke/gene_data/GSE273225.csv
Created clinical data with shape: (1, 118)
Clinical data sample:
        GSM8424381  GSM8424382  GSM8424383  GSM8424384  GSM8424385
Stroke           1           1           1           1           1
Linked data shape: (577, 118)
Linked data preview (first 5 rows, 5 columns):
        GSM8424381  GSM8424382  GSM8424383  GSM8424384  GSM8424385
Stroke         1.0         1.0         1.0         1.0         1.0
ABCB1        846.0      1291.0       662.0       577.0       394.0
ABCF1        695.0       971.0       586.0       384.0       757.0
ABL1        1013.0      1262.0       799.0       721.0       969.0
ACKR4        412.5       639.5       191.0       240.0       557.0

Dataset contains only one class for Stroke (all patients have stroke). This makes it biased for case-control studies.
Dataset deemed not usa